Connect to SQL Server

In [ ]:
from sqlalchemy import create_engine
import sqlalchemy as sqa
import urllib
ConnStr = "DRIVER = {Provide your driver name };TrustServerCertificate = yes;Authentication=ActiveDirectoryIntegrated;SERVER = GiveyourServernamewithport;DATABASE = DBNAME"
params = urllib.parse.quote_plus(ConnStr)
engine = sqa.create_engine("mssql+pyodbc:///?odbc_connect-%s"%params)
cnxn = engine.connect()
cnxn.fast_executemany = True

Update the proxy settings according to your project/requirement

In [ ]:
os.environ["HTTPS_PROXY"] = "http://yourproxy:port"
os.environ["HTTP_PROXY"] = "http://yourproxy:port"

I am using the service account. Hence reading the key from the folder where I saved

In [ ]:
os.chdir(r"C:\Users\yourname\Desktop\GCPConnector\GCP-connector")

In [ ]:
credentials = service_account.Credentials.from_service_account_file("yourserviceaccountkeyfile.json")

Connect with bigquery client using below commands

In [ ]:
project_id = "your-project-id"

Get the data from the bigquery dataset 

In [ ]:
query = '''select * from ProjectName.DatasetName'''
client = bigquery.Client(credentials= credentials,project = project_id)
query_job = client.query(query)
rows = query_job.result()
#print(rows)

Get the data into a dataframe

In [ ]:
df = rows.to_dataframe()
df.head()

Checking the number of rows in the dataframe

In [ ]:
df.shape

Function to Load huge datasets to SQL

In [ ]:
def ParallelLoad(df,TargetTable,ConnString,func,ThreadLimit = 16):
    connections = []
    for i in range(0,ThreadLimit,1):
        connections.append(ConnectToDB(ConnString))
    threads=[]
    StepSize = int(df.shape[0]/ThreadLimit)
    for i in range(0,ThreadLimit):
        process = Thread(target=func,args = [TargetTable,connections[i],df[i*StepSize:(i*StepSize)+StepSize]])
        process.start()
        threads.append(process)
    for process in threads:
        process.join()
    threads = []
    if (StepSize*ThreadLimit)!= df.shape[0]:
        process = Thread(target=func,args = [TargetTable,connection[0],df(StepSize*ThreadLimit):df.shape[0]])
        process.start()
        threads.append(process)
    for process in threads:
        process.join()
    return

Function to Create Table in SQL Database

In [ ]:
def CreateTable(df,tableName):
    columnName = list(df.column.values)
    createTableStatement = 'CREATE TABLE '+tableName + ' ('
    for i in range(len(columnName)):
        createTableStatement = createTableStatement+ '\n'+"["+columnName[i]+"]"+' '+'NVARCHAR(max) NULL'+','
    createTableStatement = createTableStatement[:-1]+')'
    print("createTableStatement ",createTableStatement)
    conn = pyodbc.connect('DRIVER = {Your Driver Name};'
                         'TrustServerCertificate = yes;'
                         'Authentication=ActiveDirectoryIntegrated;'
                          'SERVER = GiveyourServername:port;'
                          'DATABASE = DBNAME')
    cursor = conn.cursor()
    dropStatement = 'DROP TABLE IF EXIST dbo.'+tableName
    cursor.execute(dropStatement)
    cursor.execute(createTableStatement)
    logging.info(tableName+"Table Created Successfully.")
    #commit your changes in database
    conn.commit()
    #close the connection
    conn.close()

Create Table in SQL Database

In [ ]:
tableName = 'YourTableName'
CreateTable(df,tableName)

Load Data into Table

In [ ]:
result = ParallelLoad(df,tableName,ConnString,LoadToSQL,16)